# Factorized Variational Autoencoder

Following the model specified in the Disney Research paper "Factorized Variational Autoencoders for
Modeling Audience Reactions to Movies"

## Testing with toy data
We begin by testing with a toy dataset taken from the Edward examples:
https://github.com/blei-lab/edward/blob/master/examples/probabilistic_matrix_factorization.py


In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.python import debug as tf_debug

import numpy as np
import datetime
from sklearn.model_selection import train_test_split

tfd = tf.contrib.distributions

N = 1000
M = 100000
D = 3
sigma = 0.1
batch_size = N
epochs = 100

U_true = np.random.randn(D, N)
V_true = np.random.randn(D, M)

X = np.dot(np.transpose(U_true), V_true) + np.random.normal(0, sigma, size=(N, M))
X = X.astype(np.float32)


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility

/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/local/lib/python3.5/dist-packages/tensorflow/python/util/tf_inspect.py:66: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  _inspect.getargspec(target))
/usr/

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
def make_encoder(data, z_dim, batch_size, num_features):
    data = tf.reshape(data, [batch_size, num_features])

    # sample latent variables
    x = tf.layers.dense(inputs=data,
            units=512, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
            units=256, activation=tf.nn.sigmoid)
    x = tf.layers.dense(inputs=x,
            units=128, activation=tf.nn.sigmoid)
    u_net = tf.layers.dense(inputs=x,
                      units = z_dim * 2,
                      activation=None)
    u_loc = u_net[..., :z_dim]
    u_scale = tf.nn.softplus(u_net[..., z_dim:] + 0.5)
    u = tfd.MultivariateNormalDiag(u_loc, scale_diag=u_scale,
                                   name='sample_latent_U')
    
    # observation latent variables
    x_t = tf.transpose(data)
    x_t = tf.layers.dense(inputs=x_t,
            units=64, activation=tf.nn.sigmoid)
    x_t = tf.layers.dense(inputs=x_t,
            units=32, activation=tf.nn.sigmoid)
    x_t = tf.layers.dense(inputs=x_t,
            units=16, activation=tf.nn.sigmoid)
    v_net = tf.layers.dense(inputs=x_t,
                      units = z_dim * 2,
                      activation=None)
    v_loc = v_net[..., z_dim:]    
    v_scale = tf.nn.softplus(v_net[..., :z_dim] + 0.5)
    
    v = tfd.MultivariateNormalDiag(v_loc, scale_diag=v_scale,
                                   name='observation_latent_V')
    
    return u, v


def make_decoder(u, v, batch_size, num_features, z_dim):
    print(u)
    print(v)
    
    # "dot product decoder"
    z = tf.tensordot(u, v, axes=[[1], [1]])
    z = tf.reshape(z, [1, num_features*batch_size])
    
    # "neural network decoder"
#     x = tf.layers.dense(inputs=z,
#             units=128, activation=tf.nn.relu)
#     x = tf.layers.dense(inputs=x,
#              units=256, activation=tf.nn.relu)
#     x = tf.layers.dense(inputs=x,
#             units=512, activation=tf.nn.relu)
    
#     decoder_net = tf.layers.dense(inputs=x,
#             units=num_features, activation=None)
    
    # assume fixed, unit variance
    data_dist = tfd.Independent(tfd.Normal(loc=z, scale=1.,
                            name='posterior_p'))
        
    return data_dist


def make_prior(z_dim):
    u_prior =  tfd.MultivariateNormalDiag(scale_diag=tf.ones(z_dim),
                                    name='U')
    v_prior = tfd.MultivariateNormalDiag(scale_diag=tf.ones(z_dim),
                                    name='V')

    return u_prior, v_prior



In [6]:
graph = tf.Graph()
with graph.as_default():
    # input pipeline
    dataset = tf.data.Dataset.from_tensors(X)
    dataset = dataset.batch(batch_size)
    iterator = dataset.make_initializable_iterator()
    data = iterator.get_next()
    
    with tf.variable_scope('priors'):
        u_prior, u_prior = make_prior(z_dim=D)
        
    # inference network; encoder
    with tf.variable_scope('encoder'):
        u_encoder, v_encoder = make_encoder(data, z_dim=D,
                                            batch_size=batch_size,
                                            num_features=M)
    
    u = u_encoder.sample()
    v = v_encoder.sample()

    # generative network; decoder
    with tf.variable_scope('decoder'):
        decoder_p = make_decoder(u, v, z_dim=D, num_features=M,
                                 batch_size=batch_size)
    
    # prior
    with tf.variable_scope('prior'):
        u_prior, v_prior = make_prior(z_dim=D)

    # loss
    u_kl = tf.reduce_sum(tfd.kl_divergence(u_encoder, u_prior))
    v_kl = tf.reduce_sum(tfd.kl_divergence(v_encoder, v_prior))
    likelihood = tf.reduce_sum(decoder_p.log_prob(tf.reshape(data, [1, N*M])))
    elbo = -u_kl - v_kl + likelihood
    tf.summary.scalar('elbo', elbo)
    tf.summary.scalar('minus_u_kl', tf.negative(u_kl))
    tf.summary.scalar('minus_v_kl', tf.negative(v_kl))
    tf.summary.scalar('likelihood', likelihood)

    
    # optimizer
    optimizer = tf.train.AdamOptimizer(0.001).minimize(-elbo)
    merged = tf.summary.merge_all()
    

Tensor("encoder/sample_latent_U/sample/affine_linear_operator/forward/add:0", shape=(1000, 3), dtype=float32)
Tensor("encoder/observation_latent_V/sample/affine_linear_operator/forward/add:0", shape=(100000, 3), dtype=float32)


Estimate parameters of the model:

In [4]:
# tensorboard
run = 'run-{date:%d.%m.%Y_%H:%M:%S}'.format(date=datetime.datetime.now())
tb_writer = tf.summary.FileWriter('/logs/' + run, graph=graph)

# training
with tf.Session(graph=graph) as sess:    
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        sess.run(iterator.initializer)
        while True:
            try:
                _, tb_summary, epoch_elbo = sess.run([optimizer, merged, elbo])
                print(epoch_elbo)
                tb_writer.add_summary(tb_summary, epoch)
            except tf.errors.OutOfRangeError:
                break

-520124100.0
-392180320.0
-311095460.0
-272479800.0
-259612060.0
-254095730.0
-253539650.0
-251236220.0
-248798380.0
-245700100.0
-242923300.0
-240726260.0
-239169040.0
-238330930.0
-237805470.0
-237847420.0
-237943200.0
-237924530.0
-237909070.0
-237792460.0


KeyboardInterrupt: 

Now, it would be nice to add some checks for how we did.